In [1]:
import numpy as np
import pandas as pd
import gc
import pickle
import os
import datetime as dt

#plot
import matplotlib.pyplot as plt

#LightGBM
import lightgbm as lgb

#sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

#表示桁数の指定
pd.options.display.float_format = '{:.4f}'.format

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [3]:
# --- scripts パス追加 ---
import sys
if "../scripts" not in sys.path:
    sys.path.append("../scripts")

# --- utils ---
import utils
import importlib
importlib.reload(utils)
from utils import reduce_mem_usage

# --- basic_feature ---
import basic_feature
importlib.reload(basic_feature)
from basic_feature import preprocess_features

# --- predict 読み込み ---
import predict
importlib.reload(predict)
from predict import predict_lgb_regression


In [4]:
# テストデータ読み込み、メモリ削減、オブジェクトに変換
df = pd.read_csv("../data/test.csv")
df = reduce_mem_usage(df)
df = preprocess_features(df)

Memory usage decreased from 20.98 MB to 14.78 MB


In [5]:
#特徴量エンジニアリング1
# 欠損フラグを追加（1: 欠損あり、0: 欠損なし）
# 補完前のオリジナル列（欠損あり）をコピー
df['Episode_Length_minutes_raw'] = df['Episode_Length_minutes']
df['Episode_Length_minutes_was_missing'] = df['Episode_Length_minutes'].isna().astype(int)
# 欠損を中央値で補完
df['Episode_Length_minutes'] = df['Episode_Length_minutes'].fillna(df['Episode_Length_minutes'].median())

#上記と同様の操作を行う
df['Guest_Popularity_percentage_raw'] = df['Guest_Popularity_percentage']
df['Guest_Popularity_percetage_was_missing'] = df['Guest_Popularity_percentage'].isna().astype(int)
df['Guest_Popularity_percentage'] = df['Guest_Popularity_percentage'].fillna(df['Guest_Popularity_percentage'].median())




In [6]:
#データセット作成
x_test = df.drop(["id"], axis=1)
id_test = df[["id"]]

In [7]:
x_test.head()

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Episode_Length_minutes_raw,Episode_Length_minutes_was_missing,Guest_Popularity_percentage_raw,Guest_Popularity_percetage_was_missing
0,Educational Nuggets,Episode 73,78.9600,Education,38.1250,Saturday,Evening,53.3438,1.0000,Neutral,78.9600,0,53.3438,0
1,Sound Waves,Episode 23,27.8700,Music,71.3125,Sunday,Morning,53.3750,0.0000,Neutral,27.8700,0,NaN,1
2,Joke Junction,Episode 11,69.1000,Comedy,67.8750,Friday,Evening,97.5000,0.0000,Positive,69.1000,0,97.5000,0
3,Comedy Corner,Episode 73,115.3900,Comedy,23.4062,Sunday,Morning,51.7500,2.0000,Positive,115.3900,0,51.7500,0
4,Life Lessons,Episode 50,72.3200,Lifestyle,58.0938,Wednesday,Morning,11.2969,2.0000,Neutral,72.3200,0,11.2969,0


In [9]:
test_pred = predict_lgb_regression(x_test,id_test)

-------------------- Fold 0 --------------------
-------------------- Fold 1 --------------------
-------------------- Fold 2 --------------------
-------------------- Fold 3 --------------------
-------------------- Fold 4 --------------------
Inference completed.


In [11]:
df_submit = test_pred.rename(columns={"pred":"Listening_Time_minutes"})
print(df_submit.shape)
display(df_submit.head())

# ファイル出力
df_submit.to_csv("../outputs/submission_baseline.csv", index=None)

(250000, 2)


,id,Listening_Time_minutes
0,750000,54.3631
1,750001,17.9198
2,750002,49.8080
3,750003,80.8941
4,750004,48.3543
